In [ ]:

!pip install --quiet --upgrade wrds pandas psycopg2-binary
print("Installed!  Runtime → Restart runtime, then re‑run your imports.")


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 324, in run
    session = self.get_default_session(options)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 71, in get_default_session
    self._session = self.enter_context(self._build_session(options))
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/index_command.py", line 100, in _build_session
    session = PipSession(
              ^^^^^^^^^

In [ ]:

import wrds, pandas as pd, numpy as np

db = wrds.Connection()


KeyboardInterrupt: Interrupted by user

In [ ]:

sp500_idx = db.raw_sql("""
    SELECT date AS caldt,          -- keep the “caldt” alias if you like
           spindx
    FROM   crsp.dsi
    ORDER  BY date
""")


sp500_idx['caldt'] = pd.to_datetime(sp500_idx['caldt'])


sp500_idx = (sp500_idx
             .rename(columns={'spindx': 'sp500_close'})
             .set_index('caldt')
             .sort_index())

print(sp500_idx.head())

# fra 1 Jan 1985 onward
sp500_idx = sp500_idx.loc['1985-01-01':].copy()


print(sp500_idx.head(3))

NameError: name 'db' is not defined

In [ ]:
sp500_idx = (db.get_table('crsp', 'dsi', columns=['date', 'spindx'])
               .rename(columns={'spindx': 'sp500_close'})
               .set_index('date')
               .sort_index())
print(sp500_idx)

            sp500_close
date                   
1925-12-31         <NA>
1926-01-02         <NA>
1926-01-04         <NA>
1926-01-05         <NA>
1926-01-06         <NA>
1926-01-07         <NA>
1926-01-08         <NA>
1926-01-09         <NA>
1926-01-11         <NA>
1926-01-12         <NA>
1926-01-13         <NA>
1926-01-14         <NA>
1926-01-15         <NA>
1926-01-16         <NA>
1926-01-18         <NA>
1926-01-19         <NA>
1926-01-20         <NA>
1926-01-21         <NA>
1926-01-22         <NA>
1926-01-23         <NA>
1926-01-25         <NA>
1926-01-26         <NA>
1926-01-27         <NA>
1926-01-28         <NA>
1926-01-29         <NA>
1926-01-30         <NA>
1926-02-01         <NA>
1926-02-02         <NA>
1926-02-03         <NA>
1926-02-04         <NA>
1926-02-05         <NA>
1926-02-06         <NA>
1926-02-08         <NA>
1926-02-09         <NA>
1926-02-10         <NA>
1926-02-11         <NA>
1926-02-13         <NA>
1926-02-15         <NA>
1926-02-16         <NA>
1926-02-17      

In [ ]:
# 0)
sp_members = db.raw_sql("""
    SELECT permno,
           start  AS enter_date,
           COALESCE(ending, '9999-12-31') AS exit_date
    FROM   crsp.dsp500list
""")

# 1)
links = db.raw_sql("""
    SELECT lpermno AS permno,
           gvkey
    FROM   crsp.ccmxpf_linktable
    WHERE  linktype IN ('LU','LC')
      AND  linkprim IN ('P','C')
      AND  gvkey IS NOT NULL
""").drop_duplicates('permno')

# 2)
gics = db.raw_sql("""
    SELECT gvkey,
           gsector
    FROM   comp.company
    WHERE  gsector IS NOT NULL
""")

# 3)
memb_sec = (sp_members
            .merge(links, on='permno', how='left')
            .merge(gics,  on='gvkey',  how='left')
            .dropna(subset=['gsector']))

# 4)
permno_list = ', '.join(str(int(x)) for x in memb_sec['permno'].unique())

crsp_daily = db.raw_sql(f"""
    SELECT permno,
           date,
           prc,
           shrout,
           ret
    FROM   crsp.dsf
    WHERE  permno IN ({permno_list})
""")

crsp_daily['mktcap'] = crsp_daily['prc'].abs() * crsp_daily['shrout']

# 5)
crsp_daily = (crsp_daily
              .merge(memb_sec, on='permno', how='left')
              .query('enter_date <= date <= exit_date')
              .dropna(subset=['gsector']))

# 6)
sect_daily_ret = (
    crsp_daily
      .assign(weight=lambda d:
              d['mktcap'] / d.groupby(['date','gsector'])['mktcap'].transform('sum'))
      .assign(wret=lambda d: d['weight'] * d['ret'])
      .groupby(['date','gsector'])['wret'].sum()
      .reset_index()
)

# 7)
pivot_ret = (sect_daily_ret
             .pivot(index='date', columns='gsector', values='wret')
             .sort_index())

sect_idx = ((1 + pivot_ret.fillna(0)).cumprod() * 100) \
             .rename(columns={
                 '10':'Energy','15':'Materials','20':'Industrials',
                 '25':'ConsDiscr','30':'ConsStaples','35':'HealthCare',
                 '40':'Financials','45':'InfoTech','50':'CommServices',
                 '55':'Utilities','60':'RealEstate'
             }) \
             .astype(float)

# 8)
sect_idx.to_parquet('sp500_sector_indices.parquet')
print(sect_idx.head())

<ipython-input-47-9cd2518a7014>:51: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  .query('enter_date <= date <= exit_date')


gsector         Energy   Materials  Industrials   ConsDiscr  ConsStaples  \
date                                                                       
1925-12-31  100.000000  100.000000   100.000000  100.000000   100.000000   
1926-01-02  100.249933   99.607492   100.171456  102.467993    99.855071   
1926-01-04   99.540602   99.544327    99.782793  102.890419   100.887431   
1926-01-05   98.992374   98.609560    99.591721  101.660411   101.246811   
1926-01-06   98.864345   99.765705    99.687771  101.574705   101.024953   

gsector     HealthCare  Financials    InfoTech  CommServices   Utilities  \
date                                                                       
1925-12-31       100.0  100.000000  100.000000         100.0  100.000000   
1926-01-02       100.0  100.299816   99.962713         100.0  100.071159   
1926-01-04       100.0   99.967682  100.701303         100.0  100.102016   
1926-01-05       100.0   99.468037  100.232118         100.0  100.073673   
1926-01-06 

In [ ]:

sect_idx.to_csv('sp500_sector_indices.csv')
sect_idx.to_excel('sp500_sector_indices.xlsx')


from google.colab import files
files.download('/content/sp500_sector_indices.csv')
files.download('/content/sp500_sector_indices.xlsx')


NameError: name 'sect_idx' is not defined

In [ ]:

sp500_idx.to_csv('sp500_close_daily.csv')
sp500_idx.to_excel('sp500_close_daily.xlsx')


from google.colab import files
files.download('/content/sp500_close_daily.csv')
files.download('/content/sp500_close_daily.xlsx')



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>